In [1]:
import pandas as pd
import geopandas as gpd
import os

### Define helper functions

In [2]:
# Function to convert sq m to acres
m2acres = lambda x: x * 0.00024711

# Function to generate zone ID from state,
# jurisdiction name & abbreviated district name
def create_id(s, j, ad):
    s = str(s).upper()
    j = str(j).split('-')[0].strip().upper().split('/')[0]
    ad = str(ad).replace('-', '').replace(' ', '').upper()
    return f'{s}--{j}--{ad}'

### Define a function that reads and cleans up every GIS file

In [168]:
def read_zoning_file(filepath):
    try:
        # Read GIS file into dataframe
        
        gdf = (gpd
               .read_file(filepath)
               .filter(['State', 'Jurisdiction', 'AbbreviatedDistrict', 'geometry'])
               .dropna(subset=['geometry']) # remove null geometries
               .to_crs('EPSG:4326') # set projection to WGS 84 (lat/lon)
              )
        gdf.head()
        
        # Rename districts with no names to "Not Zoned"
        gdf.AbbreviatedDistrict = gdf.AbbreviatedDistrict.fillna('Not Zoned')
        
        # Create an ID column that combines jurisdiction and zoning name
        gdf['id'] = gdf.apply(
            lambda r: create_id(r.State, r.Jurisdiction, r.AbbreviatedDistrict), axis=1
        )

        # Calculate area (in acres) for each geometry
        gdf['ZoneAcres'] = (gdf
                            .to_crs('EPSG:6933') # reproject to equal area
                            .geometry.area
                            .apply(m2acres)
                           )

        # Calculate total area by zone
        total_area_by_zone = (gdf
                              .groupby('id')
                              ['ZoneAcres']
                              .sum()
                             )

        # # Combine (dissolve) geometries by zone
        gdf = gdf.dissolve(by='id')
        gdf.ZoneAcres = total_area_by_zone # assign new total areas

        # Move `id` from index to column
        gdf = gdf.reset_index()

        return gdf
               
    except Exception as e:
        print(f"Error when reading {filepath}: {e}")

### Read all GIS files using the function above

In [177]:
# Folder with all GIS files
zoning_folder = 'geojson'

# Read the folder to get all filenames (ignore hidden files)
zoning_files = [x for x in os.listdir(zoning_folder) if x[-8:] == '.geojson' ]

# Read all zones into a single pandas dataframe
gdfs = [ read_zoning_file(f"{zoning_folder}/{filename}")
         for filename in zoning_files ]

combined_df = pd.concat( gdfs ).reset_index(drop=True)

# combined_df.head()

Error when reading geojson/FederalLandsDissolve.geojson: 'GeoDataFrame' object has no attribute 'AbbreviatedDistrict'
Error when reading geojson/USAFederalLands_Montana.geojson: 'GeoDataFrame' object has no attribute 'AbbreviatedDistrict'


In [93]:
counts = combined_df.groupby('Jurisdiction').size()
counts.to_csv('counts.csv')


In [ ]:
# Set the maximum number of rows and columns to display
pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Display the full DataFrame
print(counts)


### Derive "zoneable" area for each jurisdiction by subtracting federal/state land

In [178]:
# Read federal-land GeoJSON
fed_land = gpd.read_file(
    r'geojson\USAFederalLands_Montana.geojson'
    ).to_crs(epsg=4326)

# Intersect with all zones
fed_land_ = gpd.overlay(
    combined_df,
    fed_land,
    how='intersection'
)

# Calculate acres
fed_land_['FedAcres'] = (fed_land_
    .to_crs(epsg=6933)
    .geometry.area
    .apply(m2acres)
)

# Account for repeating zone IDs
fed_land_ = (fed_land_
    .dissolve(by='id', aggfunc={'FedAcres': 'sum'})
    .reset_index()
    .filter(['id', 'FedAcres'])
)

# Add federal/state area per zoning district
combined_df = combined_df.merge(
    fed_land_,
    on='id',
    how='left'
)

# Calculate municipal (=zoneable) acres
combined_df.FedAcres = combined_df.FedAcres.fillna(0)
combined_df['MunicipalAcres'] = combined_df.ZoneAcres -\
    combined_df.FedAcres

### Read zoning data from the spreadsheet

In [171]:
spreadsheet_path = r'Zoning Atlas Montana.csv'

zoning = pd.read_csv(spreadsheet_path)\
    .loc[ :, 'Jurisdiction': 'Tooltip Notes' ]

zoning['Tooltip Notes'] = zoning['Tooltip Notes'].fillna('')

# Remove spaces from column names
zoning.columns = [x.strip() for x in zoning.columns.tolist()]

# Trim all strings in the dataframe
str_columns = zoning.select_dtypes(['object'])
zoning[ str_columns.columns ] = str_columns.apply(lambda x: x.str.strip())

# Create id column (to perform linking later)
zoning['id'] = zoning.apply(
    lambda r: create_id(r.State, r.Jurisdiction, r.AbbreviatedDistrict),
    axis=1
)

### Define a function to convert acre size to a letter category

In [163]:
# Converts min lot size into a predefined range

def min_lot_size(x):
    if x != x or x == '': # null
        return 'A'

    x = float(x)

    if x < 0.01:
        return 'A'
    if x >= 0.01 and x <= 0.0625:
        return 'B'
    if x > 0.0625 and x <= 0.125:
        return 'C'
    if x > 0.125 and x <= 0.25:
        return 'D'
    if x > 0.25 and x <= 0.5:
        return 'E'
    if x > 0.5:
        return 'F'


### Derive new measures in the zoning data
This includes "minimum lot size requirement" (true/false), "minimum unit size requirement" present (true/false), etc.

In [172]:
zoning['AduMaxSizeLimit'] = ~zoning[r'ADU Max. Size (% of MAIN UNIT)'].isna() | ~zoning['ADU Max. Size (SF)'].isna()

# Min Unit Size requirement: transform columns K, O, T, AC into true/false
zoning['1MUS'] = ~zoning['1-Family Min. Unit Size (SF)'].isna()
zoning['2MUS'] = ~zoning['2-Family Min. Unit Size (SF)'].isna()
zoning['3MUS'] = ~zoning['3-Family Min. Unit Size (SF)'].isna()
zoning['4MUS'] = ~zoning['4+-Family Min. Unit Size (SF)'].isna()

# Any minimum unit size is set? (for tooltip)
zoning['MUS'] = zoning['1MUS'] | zoning['2MUS'] | zoning['3MUS'] | zoning['4MUS']

# Min Lot Size
zoning['1MLS'] = zoning['1-Family Min. Lot (ACRES)'].apply(min_lot_size)
zoning['2MLS'] = zoning['2-Family Min. Lot (ACRES)'].apply(min_lot_size)
zoning['3MLS'] = zoning['3-Family Min. Lot (ACRES)'].apply(min_lot_size)
zoning['4MLS'] = zoning['4+-Family Min. Lot (ACRES)'].apply(min_lot_size)

# Elderly housing
# 1F Elderly Only —> value from BM column
# 2F Elderly Only —> value from BM column
# 3F Elderly Only —> Yes if BM = Yes, otherwise value from Y column
# 4F Elderly Only —> Yes if BM = Yes, otherwise value from AI column
def is_elderly_only(row, col):
    if row['Elderly Housing District'] == 'Yes':
        return 'Yes'
    return row[col]
    
zoning['1E'] = zoning['Elderly Housing District']
zoning['2E'] = zoning['Elderly Housing District']
zoning['3E'] = zoning.apply(lambda row: is_elderly_only(row, '3-Family Elderly Housing Only'), axis=1).fillna('No')
zoning['4E'] = zoning.apply(lambda row: is_elderly_only(row, '4+-Family Elderly Housing Only'), axis=1).fillna('No')

# Create ADU Elderly only
zoning['AEld'] = zoning['ADU Elderly Housing Only'].fillna('No')

In [71]:
zoning.columns

Index(['Jurisdiction', 'County', 'AbbreviatedDistrict', 'State',
       'Full District or Sub-District Name', 'District Mapped',
       'District Mapped But Extinct', 'Overlay', 'Type of Zoning District',
       'Affordable Housing District',
       ...
       'MUS', '1MLS', '2MLS', '3MLS', '4MLS', '1E', '2E', '3E', '4E', 'AEld'],
      dtype='object', length=124)

### Generate text for minimum parking requirements
This was needed for Connecticut Zoning Atals and may be optional for other atlases.

In [173]:
# Minimum parking required anywhere? for tooltip

def parking_text(row):
    one = row['1-Family Min. # Parking Spaces']
    two = row['2-Family Min. # Parking Spaces Per 2+ BR']
    two_ = row['2-Family Min. # Parking Spaces Per Studio or 1BR']
    three = row['3-Family Min. # Parking Spaces Per 2+ BR']
    three_ = row['3-Family Min. # Parking Spaces Per Studio or 1BR']
    four = row['4+-Family Min. # Parking Spaces Per Studio or 1BR']
    four_ = row['4+-Family Min. # Parking Spaces Per 2+ BR']
    
    text = []
    if one == one:
        text.append(str(one) + ' for 1-family')
    if two == two:
        text.append(str(two) + ' for 2-family (total)')
    if two_ == two_:
        text.append(str(two_) + ' for 2-family (per studio/1br)')
    if three == three:
        text.append(str(three) + ' for 3-family (total)')
    if three_ == three_:
        text.append(str(three_) + ' for 3-family (per studio/1br)')
    if four == four:
        text.append(str(four) + ' for 4+ family (total)')
    if four_ == four_:
        text.append(str(four_) + ' for 2+ family (per studio/1br)')

    return '; '.join(text)

zoning['PK'] = zoning.apply(parking_text, axis=1)

### Combine GIS and zoning data on the `id` column

In [179]:
final = combined_df.merge(
    zoning,
    on='id',
    how='left',
    suffixes=('', '_from_zoning')
)

# zoning.drop(columns=['id'], inplace=True)

In [182]:
column_types = final.dtypes.to_dict()
column_type_list = list(column_types.items())

print(column_type_list)

[('id', dtype('O')), ('geometry', <geopandas.array.GeometryDtype object at 0x0000018296F4EBC0>), ('State', dtype('O')), ('Jurisdiction', dtype('O')), ('AbbreviatedDistrict', dtype('O')), ('ZoneAcres', dtype('float64')), ('FedAcres', dtype('float64')), ('MunicipalAcres', dtype('float64')), ('Jurisdiction_from_zoning', dtype('O')), ('County', dtype('O')), ('AbbreviatedDistrict_from_zoning', dtype('O')), ('State_from_zoning', dtype('O')), ('Full District or Sub-District Name', dtype('O')), ('District Mapped', dtype('O')), ('District Mapped But Extinct', dtype('O')), ('Overlay', dtype('O')), ('Type of Zoning District', dtype('O')), ('Affordable Housing District', dtype('O')), ('Elderly Housing District', dtype('O')), ('1-Family Treatment', dtype('O')), ('2-Family Treatment', dtype('O')), ('3-Family Treatment', dtype('O')), ('4+-Family Treatment', dtype('O')), ('1-Family Min. Lot (ACRES)', dtype('float64')), ('1-Family Front Setback (# of feet)', dtype('float64')), ('1-Family Side Setback (

### Define the `overlay()` function

In [ ]:
# Given jurisdiction name and abbreviated district name,
# combine zoning regulations from base district and overlay district (has priority)
# If no base or overlay name are given, deduce from zone using `sep` as a separator between base and overlay
# For example, R-10/RM represents a base of R-10 and an overlay of RM
def overlay(j, ad, base=None, overlay=None, sep='/'):
    
    if not base:
        base = ad.split(sep)[0].strip()
        
    if not overlay:
        overlay = ad.split(sep)[1].strip()
    
    col_from = 'Type of Zoning District'
    col_to = 'PK' # Parking text column created above
    
    # Get proper zoning values from the spreadsheet
    base_values = zoning.loc[ zoning.Jurisdiction.eq(j) & zoning.AbbreviatedDistrict.eq(base), col_from : col_to ].values
    overlay_values = zoning.loc[ zoning.Jurisdiction.eq(j) & zoning.AbbreviatedDistrict.eq(overlay), col_from : col_to ].values
    
    if len(base_values) != 1:
        print(f'Base layer {base} in {j} does not exist, or is not unique!')
        return
     
    if len(overlay_values) != 1:
        print(f'Overlay layer {overlay} in {j} does not exist, or is not unique!')
        return
    
    
    base_values = base_values[0]
    overlay_values = overlay_values[0]
    
    combined_values = [ o if (o == o and o !='' and o != 'Overlay')
                            else b for b, o in zip(base_values, overlay_values) ]

    final.loc[ final.Jurisdiction.eq(j) & final.AbbreviatedDistrict.eq(ad), col_from : col_to ] = combined_values
    final.loc[ final.Jurisdiction.eq(j) & final.AbbreviatedDistrict.eq(ad), 'Jurisdiction' ] = j
    final.loc[ final.Jurisdiction.eq(j) & final.AbbreviatedDistrict.eq(ad), 'AbbreviatedDistrict' ] = base + '/' + overlay
    
    # Update full district name
    final.loc[ final['Jurisdiction'].eq(j) & final['AbbreviatedDistrict'].eq(ad), 'Full District Name' ] = zoning.loc[
        zoning.Jurisdiction.eq(j) & zoning.AbbreviatedDistrict.eq(base), 'Full District Name'].iloc[0] + '/' + zoning.loc[
        zoning.Jurisdiction.eq(j) & zoning.AbbreviatedDistrict.eq(overlay), 'Full District Name'].iloc[0]

### Run `overlay()` on each zone that has an overlay

In [ ]:
# For example:
# 
# overlay('Bristol', 'R-10/RM')
# overlay('Hartford', 'NX-1/Campus Overlay', 'NX-#', 'Campus Overlay')

### Account for non-defined zones

In [180]:
final.loc[ final.AbbreviatedDistrict.isin(['NULL', 'Not Zoned']), 'AbbreviatedDistrict' ] = 'Not Zoned'
final.loc[ final.AbbreviatedDistrict.isin(['NULL', 'Not Zoned']), 'Full District Name' ] = 'Not Zoned'
final.loc[ final.AbbreviatedDistrict.isin(['NULL', 'Not Zoned']), 'Type of Zoning District' ] = 'Nonresidential'

final.loc[ final.AbbreviatedDistrict.eq('Not Zoned'), 'MunicipalAcres' ] = 0

### Define column names to shorten

In [155]:
# Column names to shorten
cols_xwalk = {
    
    # Basic district info
    'Jurisdiction': 'T',
    'Full District or Sub-District Name': 'Z',
    'Type of Zoning District': 'Ty',
    'MunicipalAcres': 'MA',
    
    # Type of homes allowed
    '1-Family Treatment': '1F',
    '2-Family Treatment': '2F',
    '3-Family Treatment': '3F',
    '4+-Family Treatment': '4F',
    'Accessory Dwelling Unit (ADU)': 'AD',
    
    # Elderly housing
    '1E': '1E',
    '2E': '2E',
    '3E': '3E',
    '4E': '4E',
    
    # Minimum unit size requirement
    '1MUS': '1MUS',
    '2MUS': '2MUS',
    '3MUS': '3MUS',
    '4MUS': '4MUS',
    'MUS': 'MUS', # any MUS for tooltip
    
    # Mininum lot size requirement
    '1MLS': '1MLS',
    '2MLS': '2MLS',
    '3MLS': '3MLS',
    '4MLS': '4MLS',
    
    # Affordable/Elderly only
    'Affordable Housing District': 'AHD',
    'Elderly Housing District': 'EHD',
    
    # Accessory Dwelling Units restrictions
    'ADU Restricted to ONLY Primary Structure (i.e., No Outbuildings like Garages)': 'APrim',
    'AduMaxSizeLimit': 'ASize',
    'ADU Prohibition on Rental': 'ARent',
    'ADU Employees or Family Only': 'AFam',
    'ADU Owner Occupancy Required': 'AOwn',
    'AEld': 'AEld', # created above
    
    # Tooltip notes
    'Tooltip Notes': 'TN',
}

# Values to shorten
vals_xwalk = {
    'Allowed/Conditional': 'A',
    'Public Hearing': 'AH',
    'Prohibited': 'N',
    'Overlay': 'O',
    
    'Primarily Residential': 'R',
    'Mixed with Residential': 'M',
    'No Residential': 'N',
    'Nonresidential': 'N'
}

### Save as GeoJSON

This file is to be used in the web map. You can further use [`minify-geojson`](https://www.npmjs.com/package/minify-geojson) to compress the file to make it load faster.

In [181]:
(final
    .filter( list(cols_xwalk.keys()) + ['geometry'] + ['PK'])
    .rename(columns=cols_xwalk)
    .replace( vals_xwalk )
    # .assign(geometry=lambda df_: df_.geometry.simplify(0.00004))
    .to_file(r'web-map\data\final.geojson', driver='GeoJSON')
)

In [111]:
# save the zoning data to a csv
combined_df.to_csv(r'final.csv', index=False)

### Save as CSV
This file can then be used to calculate jurisdiction-level statistics and perform analysis

In [187]:
(final
 .filter([x for x in final.columns if x != 'geometry'])
 .to_csv('./final.csv')
)

# Calculations for MT Atlas Narrative

## Overall atlas statistics:
### % of zoned land evaluated that welcomes 1 family housing by-right 

In [183]:
# Filter the dataframe to only include rows where the '1-Family Treatment' column is 'Allowed/Conditional'
filtered_df = final.loc[final['1-Family Treatment'] == 'Allowed/Conditional']

# Calculate the total number of acres in the filtered dataframe
filtered_total_acres = filtered_df['ZoneAcres'].sum()

# Calculate the total number of acres in the original dataframe
total_acres = final['ZoneAcres'].sum()

# Calculate the percentage of zoned land that welcomes 1-family housing without a public hearing
percentage = (filtered_total_acres / total_acres) * 100

print(percentage)

95.49518358851277


### % of zoned land evaluated that welcomes for 2+ family housing by-right


In [184]:
# Filter the dataframe to only include rows where the '2-Family Treatment', '3-Family Treatment', or '4+-Family Treatment' columns are 'Allowed/Conditional'
filtered_df = final.loc[(final['2-Family Treatment'] == 'Allowed/Conditional') | (final['3-Family Treatment'] == 'Allowed/Conditional') | (final['4+-Family Treatment'] == 'Allowed/Conditional')]

# Calculate the total number of acres in the filtered dataframe
filtered_total_acres = filtered_df['ZoneAcres'].sum()

# Calculate the total number of acres in the original dataframe
total_acres = final['ZoneAcres'].sum()

# Calculate the percentage of zoned land that meets the specified criteria
percentage = (filtered_total_acres / total_acres) * 100

print(percentage)

50.045618321341266


### % of zoned land evaluated that welcomes ADU's

In [185]:
# Filter the dataframe to only include rows where the 'Accessory Dwelling Unit (ADU) Treatment' column is 'Allowed/Conditional'
filtered_df = final.loc[final['Accessory Dwelling Unit (ADU) Treatment'] == 'Allowed/Conditional']

# Calculate the total number of acres in the filtered dataframe
filtered_total_acres = filtered_df['ZoneAcres'].sum()

# Calculate the total number of acres in the original dataframe
total_acres = final['ZoneAcres'].sum()

# Calculate the percentage of zoned land that meets the specified criteria
percentage = (filtered_total_acres / total_acres) * 100


In [186]:
print(percentage)

80.75387407491367


### Average minimum lot size required for municipal jurisdictions evaluated

In [188]:
# Calculate the average for the "1-Family Min. Lot (ACRES)" column
average_1_family = final['1-Family Min. Lot (ACRES)'].mean()

# Print the average for the "1-Family Min. Lot (ACRES)" column
print(f'The average for the "1-Family Min. Lot (ACRES)" column is: {average_1_family}')

# Calculate the average for the "2-Family Min. Lot (ACRES)" column
average_2_family = final['2-Family Min. Lot (ACRES)'].mean()

# Print the average for the "2-Family Min. Lot (ACRES)" column
print(f'The average for the "2-Family Min. Lot (ACRES)" column is: {average_2_family}')

# Calculate the average for the "3-Family Min. Lot (ACRES)" column
average_3_family = final['3-Family Min. Lot (ACRES)'].mean()

# Print the average for the "3-Family Min. Lot (ACRES)" column
print(f'The average for the "3-Family Min. Lot (ACRES)" column is: {average_3_family}')

# Calculate the average for the "4+-Family Min. Lot (ACRES)" column
average_4plus_family = final['4+-Family Min. Lot (ACRES)'].mean()

# Print the average for the "4+-Family Min. Lot (ACRES)" column
print(f'The average for the "4+-Family Min. Lot (ACRES)" column is: {average_4plus_family}')

# Calculate the overall average
overall_average = (average_1_family + average_2_family + average_3_family + average_4plus_family) / 4

# Print the overall average
print(f'The overall average is: {overall_average}')

The average for the "1-Family Min. Lot (ACRES)" column is: 8.972729007633587
The average for the "2-Family Min. Lot (ACRES)" column is: 3.89287898089172
The average for the "3-Family Min. Lot (ACRES)" column is: 0.9933982300884956
The average for the "4+-Family Min. Lot (ACRES)" column is: 46.85892660550459
The overall average is: 15.179483206029598


## Major Cities (Missoula, Bozeman, Helena, Whitefish, Kalispell, Billings, Butte, Great Falls)
### For each city, % of zoned land that welcomes 1, 2, 3, 4 family housing by-right. % for each type. 


In [204]:
# Get a list of the desired values
desired_values = ['Missoula (City)', 'Bozeman', 'Helena', 'Whitefish', 'KALISPELL', 'Billings', 'Butte-Silver Bow (County/City)', 'Great Falls']

# Create a boolean mask indicating which values in the 'Jurisdiction' column are in the list of desired values
mask = final['Jurisdiction'].isin(desired_values)

# Use the mask to filter the original dataframe
filtered_df = final[mask]

# Group the filtered dataframe by the 'Jurisdiction' column
grouped_df = filtered_df.groupby('Jurisdiction')

# Create an empty dataframe to store the results
treatment_results_df = pd.DataFrame()

# Iterate over the groups
for name, group in grouped_df:
    # Filter the group to only include rows where the '1-Family Treatment' column is 'Allowed/Conditional'
    filtered_group_1_family = group.loc[group['1-Family Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_1_family = filtered_group_1_family['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_1_family = (filtered_total_acres_1_family / group['ZoneAcres'].sum()) * 100
    
    # Filter the group to only include rows where the '2-Family Treatment' column is 'Allowed/Conditional'
    filtered_group_2_family = group.loc[group['2-Family Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_2_family = filtered_group_2_family['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_2_family = (filtered_total_acres_2_family / group['ZoneAcres'].sum()) * 100
   
    # Filter the group to only include rows where the '3-Family Treatment' column is 'Allowed/Conditional'
    filtered_group_3_family = group.loc[group['3-Family Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_3_family = filtered_group_3_family['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_3_family = (filtered_total_acres_3_family / group['ZoneAcres'].sum()) * 100
    
    # Filter the group to only include rows where the '4+-Family Treatment' column is 'Allowed/Conditional'
    filtered_group_4plus_family = group.loc[group['4+-Family Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_4plus_family = filtered_group_4plus_family['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_4plus_family = (filtered_total_acres_4plus_family / group['ZoneAcres'].sum()) * 100
    
    # Append a row to the results dataframe
    row = pd.DataFrame({'Jurisdiction': name,
                    '1-Family Treatment': percentage_1_family,
                    '2-Family Treatment': percentage_2_family,
                    '3-Family Treatment': percentage_3_family,
                    '4+-Family Treatment': percentage_4plus_family}, index=[0])
    treatment_results_df = pd.concat([treatment_results_df, row], ignore_index=True)

treatment_results_df.head(8)

,Jurisdiction,1-Family Treatment,2-Family Treatment,3-Family Treatment,4+-Family Treatment
0,Billings,64.032165,41.175937,27.073668,28.265890
1,Bozeman,57.608427,71.652116,43.304992,43.304992
2,Butte-Silver Bow (County/City),81.492868,7.033872,6.712656,6.712656
3,Great Falls,49.955868,8.277001,9.245750,9.245750
4,Helena,66.203262,58.398965,58.076910,58.076910
5,KALISPELL,84.781329,48.339600,2.904487,2.904487
6,Missoula (City),47.372066,36.910696,28.273157,17.196003
7,Whitefish,97.554537,46.742391,31.717195,11.652107


In [210]:
# export to csv
treatment_results_df.to_csv('treatment_results.csv')

### For each city, % of zoned land that welcomes ADU's. 

In [205]:
# Get a list of the desired values
desired_values = ['Missoula (City)', 'Bozeman', 'Helena', 'Whitefish', 'KALISPELL', 'Billings', 'Butte-Silver Bow (County/City)', 'Great Falls']

# Create a boolean mask indicating which values in the 'Jurisdiction' column are in the list of desired values
mask = final['Jurisdiction'].isin(desired_values)

# Use the mask to filter the original dataframe
filtered_df = final[mask]

# Group the filtered dataframe by the 'Jurisdiction' column
grouped_df = filtered_df.groupby('Jurisdiction')

# Create an empty dataframe to store the results
adu_results_df = pd.DataFrame()

# Iterate over the groups
for name, group in grouped_df:
    # Filter the group to only include rows where the 'Accessory Dwelling Unit (ADU) Treatment' column is 'Allowed/Conditional'
    filtered_group_adu = group.loc[group['Accessory Dwelling Unit (ADU) Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_adu = filtered_group_adu['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_adu = (filtered_total_acres_adu / group['ZoneAcres'].sum()) * 100

    # Append a row to the results dataframe
    row = pd.DataFrame({'Jurisdiction': name,
                        'Accessory Dwelling Unit (ADU) Treatment': percentage_adu}, index=[0])
    adu_results_df = pd.concat([adu_results_df, row], ignore_index=True)

adu_results_df.head(8)

,Jurisdiction,Accessory Dwelling Unit (ADU) Treatment
0,Billings,17.890500
1,Bozeman,56.695686
2,Butte-Silver Bow (County/City),0.086531
3,Great Falls,64.378397
4,Helena,44.272727
5,KALISPELL,48.339600
6,Missoula (City),46.723654
7,Whitefish,72.313954


In [211]:
# export to csv
adu_results_df.to_csv('adu_results.csv')

## Major County Jurisdictions (Missoula, Gallatin, Lewis and Clark, Flathead, Yellowstone, Silver Bow, Cascade)
### For each county, % of zoned land that welcomes 1, 2, 3, 4 family housing by-right. % for each type. 


In [208]:
# Get a list of the desired values
desired_values = ['Missoula County','Gallatin', 'Lewis and Clark', 'Flathead County', 'Yellowstone County','Silver Bow County', 'Cascade County']

# Create a boolean mask indicating which values in the 'Jurisdiction' column are in the list of desired values
county_mask = final['County'].isin(desired_values)

# Use the mask to filter the original dataframe
filtered_df = final[county_mask]

# Group the filtered dataframe by the 'Jurisdiction' column
grouped_df = filtered_df.groupby('County')

# Create an empty dataframe to store the results
county_treatment_results_df = pd.DataFrame()

# Iterate over the groups
for name, group in grouped_df:
    # Filter the group to only include rows where the '1-Family Treatment' column is 'Allowed/Conditional'
    filtered_group_1_family = group.loc[group['1-Family Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_1_family = filtered_group_1_family['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_1_family = (filtered_total_acres_1_family / group['ZoneAcres'].sum()) * 100
    
    # Filter the group to only include rows where the '2-Family Treatment' column is 'Allowed/Conditional'
    filtered_group_2_family = group.loc[group['2-Family Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_2_family = filtered_group_2_family['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_2_family = (filtered_total_acres_2_family / group['ZoneAcres'].sum()) * 100
   
    # Filter the group to only include rows where the '3-Family Treatment' column is 'Allowed/Conditional'
    filtered_group_3_family = group.loc[group['3-Family Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_3_family = filtered_group_3_family['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_3_family = (filtered_total_acres_3_family / group['ZoneAcres'].sum()) * 100
    
    # Filter the group to only include rows where the '4+-Family Treatment' column is 'Allowed/Conditional'
    filtered_group_4plus_family = group.loc[group['4+-Family Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_4plus_family = filtered_group_4plus_family['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_4plus_family = (filtered_total_acres_4plus_family / group['ZoneAcres'].sum()) * 100
    
    # Append a row to the results dataframe
    row = pd.DataFrame({'Jurisdiction': name,
                    '1-Family Treatment': percentage_1_family,
                    '2-Family Treatment': percentage_2_family,
                    '3-Family Treatment': percentage_3_family,
                    '4+-Family Treatment': percentage_4plus_family}, index=[0])
    county_treatment_results_df = pd.concat([county_treatment_results_df, row], ignore_index=True)

county_treatment_results_df.head(8)

,Jurisdiction,1-Family Treatment,2-Family Treatment,3-Family Treatment,4+-Family Treatment
0,Cascade County,99.114737,91.781163,0.170240,0.170240
1,Flathead County,82.109019,14.770707,13.153985,12.909112
2,Gallatin,78.291101,25.388515,21.551944,23.191003
3,Lewis and Clark,89.386865,88.483032,2.971166,2.971166
4,Missoula County,90.775024,53.798195,6.808844,5.274256
5,Silver Bow County,81.492868,7.033872,6.712656,6.712656
6,Yellowstone County,81.915773,10.874892,7.365457,7.389343


In [212]:
# export to csv
county_treatment_results_df.to_csv('county_treatment_results.csv')

### For each county, % of zoned land that welcomes ADU's. % for each county. 

In [209]:
# Get a list of the desired values
desired_values = ['Missoula County','Gallatin', 'Lewis and Clark', 'Flathead County', 'Yellowstone County','Silver Bow County', 'Cascade County']

# Create a boolean mask indicating which values in the 'Jurisdiction' column are in the list of desired values
county_mask = final['County'].isin(desired_values)

# Use the mask to filter the original dataframe
filtered_df = final[county_mask]

# Group the filtered dataframe by the 'Jurisdiction' column
grouped_df = filtered_df.groupby('County')

# Create an empty dataframe to store the results
county_adu_results_df = pd.DataFrame()

# Iterate over the groups
for name, group in grouped_df:
    # Filter the group to only include rows where the 'Accessory Dwelling Unit (ADU) Treatment' column is 'Allowed/Conditional'
    filtered_group_adu = group.loc[group['Accessory Dwelling Unit (ADU) Treatment'] == 'Allowed/Conditional']
    # Calculate the total number of acres in the filtered group
    filtered_total_acres_adu = filtered_group_adu['ZoneAcres'].sum()
    # Calculate the percentage of zoned land in the jurisdiction that meets the specified criteria
    percentage_adu = (filtered_total_acres_adu / group['ZoneAcres'].sum()) * 100

    # Append a row to the results dataframe
    row = pd.DataFrame({'Jurisdiction': name,
                        'Accessory Dwelling Unit (ADU) Treatment': percentage_adu}, index=[0])
    county_adu_results_df = pd.concat([county_adu_results_df, row], ignore_index=True)

county_adu_results_df.head(8)

,Jurisdiction,Accessory Dwelling Unit (ADU) Treatment
0,Cascade County,92.267722
1,Flathead County,31.765063
2,Gallatin,66.513132
3,Lewis and Clark,10.396394
4,Missoula County,90.436949
5,Silver Bow County,0.086531
6,Yellowstone County,54.474881


In [213]:
# export to csv
county_adu_results_df.to_csv('county_adu_results.csv')